In [40]:
import random
import json
import numpy as np
import argparse
import base64

In [27]:
# Configuration Variables
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
padding = 50
SEGMENTATION_LENGTH = 10
MAX_NUMBER_OF_ANNOTATIONS = 10

In [62]:
# Helper functions
def bounding_box_from_points(points):
    """
        This function only supports the `poly` format.
    """
    points = np.array(points).flatten()
    even_locations = np.arange(points.shape[0]/2) * 2
    odd_locations = even_locations + 1
    X = np.take(points, even_locations.tolist())
    Y = np.take(points, odd_locations.tolist())
    bbox = [X.min(), Y.min(), X.max()-X.min(), Y.max()-Y.min()]
    bbox = [int(b) for b in bbox]
    return bbox

# Submission Format

The grader expects you to submit a JSON file which is a list of multiple `annotation` objects.

A single annotation object has the following structure :

```javascript
annotation = {
    "image_id" : 13452,
    "category_id" : 100,
    "bbox" : [32,32,62,62],
    "segmentation" : [[
             94,
             78,
             32,
             78,
             32,
             25,
             94,
             25,
             94,
             78
          ]],
    "score" : 0.86          
}
```
where all the parameters have the same meaning as explained in the [Parsing Annotations](https://github.com/crowdAI/mapping-challenge-starter-kit/blob/master/Dataset%20Utils.ipynb) section. The only new key is the `score` key, which represents your confidence in the said annotation. You can submit upto 100 such annotations for every image in the test set.

The final submitted JSON will have the following structure : 
```json
predictions = [annotation_1, annotation_2, annotation_3, annotation_4, ...]
```

## Generate a random segmentation
Now lets begin by trying to define a function which can provide us a single segmentation object given an image :


In [63]:
def single_segmentation(image_width, image_height, number_of_points=10):
    points = []
    for k in range(number_of_points):
        # Choose a random x-coordinate
        random_x = int(random.randint(0, image_width))
        # Choose a random y-coordinate
        random_y = int(random.randint(0, image_height))
        #Flatly append them to the list of points
        points.append(random_x)
        points.append(random_y)
    return [points]

## Generate a random annotation object
Now we can try to define a function which can generate a single annotation object as expected by the grader :

In [64]:
def single_annotation(image_id, number_of_points=10):
    _result = {}
    _result["image_id"] = image_id
    _result["category_id"] = 100 # as 100 is the category_id of Building
    _result["score"] = np.random.rand() # a random score between 0 and 1
    
    _result["segmentation"] = single_segmentation(IMAGE_WIDTH, IMAGE_HEIGHT, number_of_points=number_of_points)
    _result["bbox"] = bounding_box_from_points(_result["segmentation"])
    return _result

In [65]:
single_annotation(1009)

{'bbox': [2, 4, 297, 157],
 'category_id': 100,
 'image_id': 1009,
 'score': 0.752167211952941,
 'segmentation': [[298,
   149,
   193,
   140,
   299,
   26,
   225,
   161,
   102,
   49,
   194,
   16,
   274,
   78,
   2,
   151,
   163,
   13,
   89,
   4]]}

## Generate a results object

Now, we will try to use the functions created above to create a random number of annotations for all the images in the test set.   

We know that the test set has a total of `60697` images, and their image_ids are sequentially numbered.
so we can basically generate all the image_ids in the test set by doing : 
```python
IMAGE_IDS = list(range(60697)) 
```

In [66]:
IMAGE_IDS = list(range(60697)) 


Now we iterate over all the `image_ids`, generate an arbitrary number of annotations for every image, and create a final results object.

In [67]:
predictions = []
for image_id in IMAGE_IDS:
    number_of_annotations = random.randint(0, MAX_NUMBER_OF_ANNOTATIONS)
    for _idx in range(number_of_annotations):
        _annotation = single_annotation(image_id)
        predictions.append(_annotation)

## Write the results object

In [71]:
import json
fp = open("predictions.json", "w")
fp.write(json.dumps(predictions))
fp.close()

# Submit to crowdAI for grading

**Note** Please do remember to replace `"YOUR CROWDAI API KEY HERE"` with your crowdAI API Key from     
`https://www.crowdai.org/participants/<your-crowdai-username>`

In [ ]:
!pip install -U crowdai

In [ ]:
import crowdai
api_key = "YOUR CROWDAI API KEY HERE"

challenge = crowdai.Challenge("crowdAIMappingChallenge", api_key)
result = challenge.submit("predictions.json")
print(result)